## Simple faults - Attacker model 1 

Here we start to play around with the fault attacks on CSIDH.

In [1]:
from csidh import CSIDH
import chipwhisperer as cw
import time

In [2]:
PATH = '/home/tjaros/Documents/thesis/csidh-target/src/'
csidh = CSIDH(PATH)

In [3]:
csidh.setup()

In [5]:
csidh.flash_target()

simpleserial/Makefile.simpleserial:24: *** Invalid SimpleSerial version: {self.SS_VER}; allowed verions: 
  +---------+--------------+
  | Version | SS_VER value |
  +---------+--------------+
  | V1.0    | SS_VER_1_0   |
  | V1.1    | SS_VER_1_1   |
  | V2.1    | SS_VER_2_1   |
  +---------+--------------+
.  Stop.


SS_VER set to SS_VER_2_1
rm -f -- main-CWLITEARM.hex
rm -f -- main-CWLITEARM.eep
rm -f -- main-CWLITEARM.cof
rm -f -- main-CWLITEARM.elf
rm -f -- main-CWLITEARM.map
rm -f -- main-CWLITEARM.sym
rm -f -- main-CWLITEARM.lss
rm -f -- objdir/*.o
rm -f -- objdir/*.lst
rm -f -- main.s csidh.s fp.s mont.s randombytes.s uint.s parametrization.s simpleserial.s stm32f3_hal.s stm32f3_hal_lowlevel.s stm32f3_sysmem.s
rm -f -- main.d csidh.d fp.d mont.d randombytes.d uint.d parametrization.d simpleserial.d stm32f3_hal.d stm32f3_hal_lowlevel.d stm32f3_sysmem.d
rm -f -- main.i csidh.i fp.i mont.i randombytes.i uint.i parametrization.i simpleserial.i stm32f3_hal.i stm32f3_hal_lowlevel.i stm32f3_sysmem.i
.
-------- begin --------
arm-none-eabi-gcc (Arm GNU Toolchain 12.2.Rel1 (Build arm-12.24)) 12.2.1 20221205
Copyright (C) 2022 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPO

/nix/store/8w6rddmpg7b1xg2sysjq7kz0aqfpvw12-gcc-arm-embedded-12.2.rel1/bin/../lib/gcc/arm-none-eabi/12.2.1/../../../../arm-none-eabi/bin/ld: /nix/store/8w6rddmpg7b1xg2sysjq7kz0aqfpvw12-gcc-arm-embedded-12.2.rel1/bin/../lib/gcc/arm-none-eabi/12.2.1/../../../../arm-none-eabi/lib/thumb/v7e-m/nofp/libg_nano.a(libc_a-closer.o): in function `_close_r':
closer.c:(.text._close_r+0xc): warning: _close is not implemented and will always fail
/nix/store/8w6rddmpg7b1xg2sysjq7kz0aqfpvw12-gcc-arm-embedded-12.2.rel1/bin/../lib/gcc/arm-none-eabi/12.2.1/../../../../arm-none-eabi/bin/ld: /nix/store/8w6rddmpg7b1xg2sysjq7kz0aqfpvw12-gcc-arm-embedded-12.2.rel1/bin/../lib/gcc/arm-none-eabi/12.2.1/../../../../arm-none-eabi/lib/thumb/v7e-m/nofp/libg_nano.a(libc_a-fstatr.o): in function `_fstat_r':
fstatr.c:(.text._fstat_r+0xe): warning: _fstat is not implemented and will always fail
/nix/store/8w6rddmpg7b1xg2sysjq7kz0aqfpvw12-gcc-arm-embedded-12.2.rel1/bin/../lib/gcc/arm-none-eabi/12.2.1/../../../../arm-none-

Detected known STMF32: STM32F302xB(C)/303xB(C)
Extended erase (0x44), this can take ten seconds or more
Attempting to program 15259 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 15259 bytes


In [4]:
cw.scope_logger.setLevel(cw.logging.WARNING)

In [5]:
csidh.scope.clock.adc_src = 'clkgen_x1'
csidh.reset_target()
csidh.scope.clock

adc_src       = clkgen_x1
adc_phase     = 0
adc_freq      = 29538459
adc_rate      = 147692.295
adc_locked    = True
freq_ctr      = 0
freq_ctr_src  = extclk
clkgen_src    = system
extclk_freq   = 10000000
clkgen_mul    = 2
clkgen_div    = 26
clkgen_freq   = 7384615.384615385
clkgen_locked = True

In [9]:
csidh.reset_target()
csidh.private = [4, 0, 0]
csidh.public  = 0
csidh.action()
csidh.private = [5, 0, 0]
csidh.scope.arm()
csidh.action()
ret = csidh.scope.capture()
if ret:
    print("Timeout happened during acquisition")

public_correct = csidh.public_with_errors
print(public_correct)

413


In [10]:
print(csidh.scope.adc.trig_count)

2186467


In [11]:
csidh.scope.glitch.clk_src = 'clkgen'
csidh.scope.glitch.output = "clock_xor"
csidh.scope.glitch.trigger_src = "ext_single"
csidh.scope.glitch.repeat = 5
# These width/offset numbers are for CW-Lite/Pro; for CW-Husky, convert as per Fault 1_1:
csidh.scope.glitch.width = 6
csidh.scope.glitch.offset = -13
csidh.scope.glitch.ext_offset = 70000
csidh.scope.io.hs2 = "glitch"
csidh.scope.glitch

clk_src     = clkgen
width       = 5.859375
width_fine  = 0
offset      = -12.890625
offset_fine = 0
trigger_src = ext_single
arm_timing  = after_scope
ext_offset  = 70000
repeat      = 5
output      = clock_xor

In [12]:
from tqdm.notebook import trange
import random

In [13]:
results = []
f = open("gliches.csv", "w")
f.write('"scope.glitch.width", "scope.glitch.offset", "scope.glitch.repeat", "scope.glitch.ext_offset", "good/bad/crash\n"')
f.flush()

for i in trange(1, 600):
    csidh.scope.glitch.ext_offset = int(random.uniform(1, 2200000))
    csidh.scope.adc.timeout = 5
    csidh.reset_target()
    csidh.public  = 0
    csidh.private = [9, 0, 0]
    csidh.scope.arm()
    csidh.action()
    ret = csidh.scope.capture()
    ret = csidh.scope.capture()
    if ret:
        print("Timeout happened during acquisition")
        continue

    public_received = csidh.public_with_errors
    
    if not isinstance(public_received, int):
        result = "crash"
    else:
        result = "good" if public_received == public_correct else "bad"
    

    f.write(
        f"{csidh.scope.glitch.width},{csidh.scope.glitch.offset},{csidh.scope.glitch.repeat},{result}\n"
    )
    f.flush()



    

  0%|          | 0/599 [00:00<?, ?it/s]

(ChipWhisperer Scope ERROR|File OpenADC.py:787) Received fewer points than expected: 24399 vs 24400


Timeout happened during acquisition


(ChipWhisperer Scope ERROR|File OpenADC.py:787) Received fewer points than expected: 24399 vs 24400


Timeout happened during acquisition


In [ ]:
first_iso_offset = 76000